In [1]:
import pandas as pd
import json
import pymysql
config = {
    'host': 'localhost',
    'port': 3306,
    'user': 'root',
    'password': '123456',
    'database': 'yuemaifang',
    'charset': 'utf8mb4',
    'cursorclass': pymysql.cursors.Cursor
}

def con_sql(sql):
	db = pymysql.connect(**config)
	cursor = db.cursor()
	cursor.execute(sql)
	result = cursor.fetchall()
	df = pd.DataFrame(list(result))
	db.close()
	return df

In [2]:
data = con_sql('select * from data')
data

,0,1,2,3,4,5,6,7,8,9
0,湛江渔人码头,data,湛江市,2018年05月30日开盘,电话：400-7055-050 转 11780,霞山 霞山 · 东堤路15号,在售,商业,60000,小户型 综合体 海景
1,江门美吉特广场CCMALL,data,江门市,2016年03月30日开盘,电话：400-6378-135 转 466,新会区 · 启超大道9号,在售,商业,30000,品牌地产 综合体 景观居所
2,尚东柏悦府,data,广州市,2016年10月08日开盘,电话：400-6390-343 转 1409,天河 珠江新城 · 珠江新城花城大道98号（珠江公园南门对面）,在售,住宅,160000,地铁沿线
3,益田大运城邦别墅,data,东莞市,2014年03月29日开盘,电话：400-7055-050 转 17149,常平 常平 · 东莞市凤岗道龙平路官井头村,在售,别墅,60000,景观居所
4,华鸿璟悦轩,data,中山市,2021年05月01日开盘,电话：400-6372-095 转 119,火炬开发区 凯茵新城 · 博爱七路与景观路交汇处长江高尔夫球场旁,在售,住宅,30000,品牌地产
...,...,...,...,...,...,...,...,...,...,...
6953,汇生商贸大楼,data,珠海市,1970年01月01日开盘,电话：400-6390-473 转 1001,香洲 · 前山金鸡路125号（人力资源大厦对面）,售完,商业,售价待定,现房
6954,文德广场,data,珠海市,1970年01月01日开盘,电话：400-6390-473 转 1071,高栏港经济区 · 平沙镇美平西路328号文德广场销售中心,售完,住宅,售价待定,品牌地产 配套成熟
6955,奥园阳光一号,data,珠海市,2019年06月22日开盘,电话：400-6390-473 转 1108,金湾 · 三灶镇机场西路101号,售完,住宅,售价待定,小户型
6956,泉福豪亭,data,珠海市,1970年01月01日开盘,电话：400-6390-473 转 1006,香洲 · 凤凰北路2000号（翠香路交叉口）,售完,商业,售价待定,现房


In [5]:
data.columns = ['楼盘名', 'data', '市', '开盘日期', '联系电话', '地址', '售卖状态', '房产类型', '售价', 'Tag']
# data.drop('data', axis=1, inplace=True)

In [6]:
df1 = data[data['售卖状态'] == '待售']
df2 = df1[df1['售价'] != '售价待定']
df3 = df2['市'].value_counts()
shi1 = df3.index.tolist()
i = []
j = []
for a in shi1:
    h = a.strip('市')
    b = df2[df2['市'] == a]
    c = b['售价'].values.tolist()
    e = 0
    for d in c:
        e += int(d)
    f = len(c)
    g = e / f
    j.append(str(h))
    i.append(str(round(g, 2)))

In [8]:
pricing = {
    "city": j,
    "price": i
}
pricing_list = list(zip(pricing['city'], pricing['price']))
pricing_list.sort(key=lambda x: float(x[1]), reverse=True)
result = {
    "name": [i[0] for i in pricing_list],
    "value": [i[1] for i in pricing_list]
}


{'city': ['深圳', '惠州', '广州', '佛山', '东莞', '江门', '珠海', '中山', '湛江', '河源', '清远', '阳江', '韶关', '茂名', '肇庆', '云浮'], 'price': ['64500.91', '13105.51', '40456.06', '17556.67', '26952.95', '11163.87', '28209.68', '13125.93', '9494.55', '6591.08', '7990.0', '5460.0', '5975.0', '6342.86', '8437.6', '4000.0']}
[('深圳', '64500.91'), ('广州', '40456.06'), ('珠海', '28209.68'), ('东莞', '26952.95'), ('佛山', '17556.67'), ('中山', '13125.93'), ('惠州', '13105.51'), ('江门', '11163.87'), ('湛江', '9494.55'), ('肇庆', '8437.6'), ('清远', '7990.0'), ('河源', '6591.08'), ('茂名', '6342.86'), ('韶关', '5975.0'), ('阳江', '5460.0'), ('云浮', '4000.0')]
1. 深圳 64500.91元
2. 广州 40456.06元
3. 珠海 28209.68元
4. 东莞 26952.95元
5. 佛山 17556.67元
6. 中山 13125.93元
7. 惠州 13105.51元
8. 江门 11163.87元
9. 湛江 9494.55元
10. 肇庆 8437.6元
11. 清远 7990.0元
12. 河源 6591.08元
13. 茂名 6342.86元
14. 韶关 5975.0元
15. 阳江 5460.0元
16. 云浮 4000.0元
